In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images to range [-1, 1]
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)


Files already downloaded and verified


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
resnet_model = models.resnet18(pretrained=False)
resnet_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_model.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
class ResNetModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.resnet = resnet_model
    self.dense_head = nn.Sequential(
        nn.Linear(1000, 10),
        nn.Softmax(),
    )
  def forward(self, x):
    x = self.resnet(x)
    x = self.dense_head(x)
    return x

In [6]:
resnetmodel = ResNetModel().to(device)

In [ ]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), F.one_hot(data[1].to(device), num_classes=10)
        labels = labels.type(torch.FloatTensor).to(device)


        optimizer.zero_grad()

        outputs = resnetmodel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # Print every 200 mini-batches
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 200:.3f}")
            running_loss = 0.0
            print(f"Output dimensions: {outputs.size()}\nLabel dimensions: {labels.size()}")

print("Finished Training ResNet18 on CIFAR-10")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[Epoch 1, Batch 200] Loss: 2.286
Output dimensions: torch.Size([64, 10])
Label dimensions: torch.Size([64, 10])
[Epoch 1, Batch 400] Loss: 2.229
Output dimensions: torch.Size([64, 10])
Label dimensions: torch.Size([64, 10])
[Epoch 1, Batch 600] Loss: 2.175
Output dimensions: torch.Size([64, 10])
Label dimensions: torch.Size([64, 10])
[Epoch 2, Batch 200] Loss: 2.092
Output dimensions: torch.Size([64, 10])
Label dimensions: torch.Size([64, 10])
[Epoch 2, Batch 400] Loss: 2.054
Output dimensions: torch.Size([64, 10])
Label dimensions: torch.Size([64, 10])
[Epoch 2, Batch 600] Loss: 2.043
Output dimensions: torch.Size([64, 10])
Label dimensions: torch.Size([64, 10])
[Epoch 3, Batch 200] Loss: 1.999
Output dimensions: torch.Size([64, 10])
Label dimensions: torch.Size([64, 10])
[Epoch 3, Batch 400] Loss: 1.983
Output dimensions: torch.Size([64, 10])
Label dimensions: torch.Size([64, 10])
[Epoch 3, Batch 600] Loss: 1.973
Output dimensions: torch.Size([64, 10])
Label dimensions: torch.Size([6

In [ ]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True, num_workers=2)

In [ ]:
correct = 0
total = 0
for i, data in enumerate(testloader, 0):
        inputs, labels = data[0].to(device), F.one_hot(data[1].to(device))
        outputs = resnetmodel(inputs)
        if(i==0):
          print(f"Output dimensions: {outputs.size()}\nLabel dimensions: {labels.size()}")
        outputs = torch.argmax(outputs, dim=1)
        labels = torch.argmax(labels, dim=1)
        correct += torch.sum(torch.eq(outputs, labels))
        total += 64
print(f"Test accuracy: {correct/total*100:.2f}%")
